In [1]:
import json

with open('review_list_그랜드 하얏트 제주.json','r',encoding='UTF8') as f:
    review_list = json.load(f)

In [2]:
good_cnt, bad_cnt = 0, 0

for review in review_list:
    if review['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

In [3]:
reviews_good, reviews_bad = [], []

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

In [4]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='UTF8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews("review_list_그랜드 하얏트 제주.json")

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

## Zero-shot

In [12]:
prompt = f"""아래 숙소에 대해서 3문장 내로 요약해줘:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt).choices[0].message.content}")

좋은 리뷰 요약: 1. 전망과 뷔페식당이 좋았지만 상점이 부족한 숙소
2. 수영하고 싶어서 선택한 호텔이지만 수영장 음식물 섭취가 불가능한 불편함 있음
3. 최고의 호텔 시설이지만 중국인 관광객이 많아서 소음과 불편함 있음
나쁜 리뷰 요약: 1. 중국인들이 많아서 냄새가 나는 호텔이지만 객실과 침구는 만족스러웠고 조식은 별로였다.
2. 호텔은 깨끗하고 좋았지만 직원의 친절도가 불균형하며 한국인 투숙객이 없어서 아쉬웠다.
3. 오션뷰와 서비스는 좋았지만 화재 발생 시 안내가 미흡하고 중국인이 많아서 불편한 점이 있었다.


## One-Shot

In [13]:
summary_good = summarize(good, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content
summary_bad = summarize(bad, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content

In [14]:
prompt_1shot = f"""아래 숙소에 대해서 3문장 내로 요약해줘:
다음은 요약 예시입니다.
예시 요약 결과:{summary_good}
"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_1shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_1shot).choices[0].message.content}")

좋은 리뷰 요약: 이 숙소는 넓은 객실과 좋은 전망, 뷔페식당이 좋은 평가를 받고 있으며, 주변 상점이 부족하다는 의견이 있습니다. 또한, 수영장과 야외수영장에서 바라보는 바다와 산뷰가 좋다는 평가를 받고 있습니다. 중국인 관광객이 많고, 새벽 소음과 서비스 면에서 개선이 필요하다는 의견도 있습니다.
나쁜 리뷰 요약: 이 숙소는 중국인 투숙객이 많고, 일부 서비스 면에서 개선이 필요하다는 의견이 있지만, 객실과 침구는 만족스럽고 깨끗하며, 오션뷰와 서비스는 좋다는 평가를 받고 있습니다. 다만, 조식이나 부대시설 안내가 미흡하다는 의견도 있으며, 중국인이 많아 한국인 투숙객이 부족하다는 점이 아쉽다고 합니다.


## Two-Shot

In [15]:
prompt_2shot = f"""아래 숙소에 대해서 3문장 내로 요약해줘:
다음은 요약 예시입니다.
좋은 리뷰 요약 결과:
{summary_good}
나쁜 리뷰 요약 결과:
{summary_bad}
"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_2shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_2shot).choices[0].message.content}")

좋은 리뷰 요약: 1. 뷰와 서비스가 좋고, 수영장 이용이 편리한 호텔이지만 중국인 투숙객이 많아 소음이 있을 수 있습니다.
2. 고급스러운 객실과 좋은 뷰, 수영장을 갖춘 호텔로, 중국어 안내문과 중국인 손님이 많아 한국인에게는 불편할 수 있습니다.
3. 넓은 객실과 좋은 뷰, 수영장을 갖춘 호텔로, 직원 서비스와 중국인 손님이 많아 개선이 필요한 부분이 있습니다.
나쁜 리뷰 요약: 1. 중국인 투숙객이 많아 냄새와 소음 문제가 있으며, 주변 환경도 중국인 중심으로 형성되어 있어 한국인에게는 불편할 수 있습니다.
2. 객실은 깨끗하고 만족스럽지만 직원 친절도와 안내 서비스가 부족하다는 지적이 있습니다.
3. 조식 포함으로 예약한 줄 알았는데 불포함이었고, 수영장이 좁고 풀바가 없어서 아쉬움을 표현한 리뷰가 있습니다.
